<a href="https://colab.research.google.com/github/yazhcodes/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Set Spark Version
import os
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Java, Spark and PostgreSQL Driver
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

# Store DB Server Password
from getpass import getpass
password = getpass('Enter DB Server password')

# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-1.cmvclswi6ved.us-east-2.rds.amazonaws.com:5432/AmazonVineDB"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [61.8 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [ ]:
# Import Dependencies
from pyspark import SparkFiles
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import round

# **Extraction**

In [ ]:
# Read Raw Data
url ="https://yazhcodes-amozon-vine.s3.us-east-2.amazonaws.com/amazon_reviews_us_Home_Improvement_v1_00.tsv"
spark.sparkContext.addFile(url)
raw_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Home_Improvement_v1_00.tsv"), sep="\t", header=True)
raw_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48881148|R215C9BDXTDQOW|B00FR4YQYK|     381800308|SadoTech Model C ...|Home Improvement|          4|            0|          0|   N|                Y|          Four Stars|        good product| 2015-08-31|
|         US|   47882936|R1DTPUV1J57YHA|B00439MYYE|     921341748|iSpring T32M 3.2 ...|Home Improvement|          5|    

# **Transformation**

In [ ]:
type_cast_df = raw_df.withColumn('star_rating',raw_df['star_rating'].cast(IntegerType()))
type_cast_df = type_cast_df.withColumn('helpful_votes',type_cast_df['helpful_votes'].cast(IntegerType()))
type_cast_df = type_cast_df.withColumn('total_votes',type_cast_df['total_votes'].cast(IntegerType()))
type_cast_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
# Vine Dataframe
vine_df = type_cast_df.select([
                               'review_id',
                               'star_rating',
                               'helpful_votes',
                               'total_votes',
                               'vine',
                               'verified_purchase'
                               ])
vine_df.show()
vine_df.printSchema()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R215C9BDXTDQOW|          4|            0|          0|   N|                Y|
|R1DTPUV1J57YHA|          5|            0|          0|   N|                Y|
| RFAZK5EWKJWOU|          5|            0|          0|   N|                Y|
|R2XT8X0O0WS1AL|          5|            0|          0|   N|                Y|
|R14GRNANKO2Y2J|          5|            0|          0|   N|                Y|
|R2BLF9VYL24LCQ|          5|            1|          1|   N|                Y|
|R1GI9UW5KJ671O|          5|            0|          0|   N|                Y|
|R2H5CEJN863M86|          5|            0|          1|   N|                Y|
| R5PPDHFOZ3SMU|          5|            0|          0|   N|                Y|
| RE1L9IENKJJ7Y|          1|            0|          0|   N|     

In [ ]:
# Filter based on votes
votes_filter_df = vine_df.\
                filter(vine_df['total_votes']>=20).\
                filter(vine_analysis_df1['helpful_votes']/vine_analysis_df1['total_votes']>=0.5)
votes_filter_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2VIQ3UX794Q0O|          4|           21|         21|   N|                Y|
|R1OA24IIHWF54G|          5|           43|         45|   N|                Y|
| RJ7N3OOJR9RL0|          4|           63|         71|   N|                Y|
|R1W8778CBXSRU6|          5|           28|         29|   N|                Y|
|R2EFAM03SWLIJX|          1|           32|         35|   N|                Y|
|R3F8P56ZDJ6PI8|          4|          106|        115|   N|                Y|
|R2QYLQEK9UAJQ6|          5|           33|         34|   N|                Y|
| RQCC7XNYI014B|          1|           91|         92|   N|                Y|
|R2U8FRHRVW5D2C|          5|           23|         23|   N|                N|
|R3RJZC5J4VP7AT|          4|           86|         91|   N|     

In [ ]:
# Aggregate all reviews by Vine
all_grouped_df = votes_filter_df.\
                          groupby('vine').\
                          agg({'review_id':'count'})
all_grouped_df = all_grouped_df.withColumnRenamed('count(review_id)','all_reviews')
all_grouped_df.show()

+----+-----------+
|vine|all_reviews|
+----+-----------+
|   Y|        266|
|   N|      38829|
+----+-----------+



In [ ]:
# Aggregate the 5 star reviews by Vine
five_star_grouped_df = votes_filter_df.\
                          filter(votes_filter_df['star_rating']==5).\
                          groupby('vine').\
                          agg({'review_id':'count'})
five_star_grouped_df = five_star_grouped_df.withColumnRenamed('count(review_id)','five_star_reviews')
five_star_grouped_df.show()

+----+-----------------+
|vine|five_star_reviews|
+----+-----------------+
|   Y|              125|
|   N|            18246|
+----+-----------------+



In [ ]:
# Join the aggregated dataframes
vine_analysis_df = all_grouped_df.join(five_star_grouped_df,on='vine',how='inner')
vine_analysis_df.show()

+----+-----------+-----------------+
|vine|all_reviews|five_star_reviews|
+----+-----------+-----------------+
|   Y|        266|              125|
|   N|      38829|            18246|
+----+-----------+-----------------+



In [ ]:
# Calculate Five Star Rating Percentages for Vine and Non-Vine reviews
vine_analysis_df = vine_analysis_df.withColumn('five_star_reviews_percentage',round(vine_analysis_df['five_star_reviews']/vine_analysis_df['all_reviews']*100,2))
vine_analysis_df.show()

+----+-----------+-----------------+----------------------------+
|vine|all_reviews|five_star_reviews|five_star_reviews_percentage|
+----+-----------+-----------------+----------------------------+
|   Y|        266|              125|                       46.99|
|   N|      38829|            18246|                       46.99|
+----+-----------+-----------------+----------------------------+

